In [1]:
import yaml
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine, text
import pandas as pd

print("Todas las librerías se importaron correctamente")


Todas las librerías se importaron correctamente


In [2]:
def load_config(file_path="config.yaml"):
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

In [8]:
config = load_config()
db_config = config["database"]

# Load credentials
db_user = db_config["user"]
db_password = db_config["password"]
db_host = db_config["host"]
db_port = db_config["port"]
db_name = db_config["name"]

In [9]:
# DB connection
conn = psycopg2.connect(
    dbname="postgres",
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port,

)
conn.autocommit = True

In [12]:
db_name = "postgres"
try:
    with conn.cursor() as cur:
        cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
        print(f"Base de datos '{db_name}' creada exitosamente.")
except psycopg2.errors.DuplicateDatabase:
    print(f"La base de datos '{db_name}' ya existe.")
finally:
    conn.close()

La base de datos 'postgres' ya existe.


In [ ]:
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS tabla_etl (
            id SERIAL PRIMARY KEY,
            nombre VARCHAR(100),
            edad INT,
            email VARCHAR(100),
            fecha_registro TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """))
    conn.commit()  # Asegúrate de confirmar los cambios
    print("Tabla 'tabla_etl' creada exitosamente en PostgreSQL.")

Tabla 'tabla_etl' creada exitosamente en PostgreSQL.


In [25]:
from sqlalchemy import text

# Insertar datos en la tabla
with engine.connect() as conn:
    conn.execute(text("""
        INSERT INTO tabla_etl (nombre, edad, email)
        VALUES 
            ('Juan Pérez', 28, 'juan@example.com'),
            ('María García', 34, 'maria@example.com'),
            ('Carlos López', 25, 'carlos@example.com');
    """))
    conn.commit()  # Confirmar los cambios
    print("Datos insertados exitosamente en 'tabla_etl'.")

Datos insertados exitosamente en 'tabla_etl'.


In [26]:
# Leer datos de la tabla
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM tabla_etl;"))
    rows = result.fetchall()

# Mostrar los resultados
print("Datos en 'tabla_etl':")
for row in rows:
    print(row)

Datos en 'tabla_etl':
(1, 'Juan Pérez', 28, 'juan@example.com', datetime.datetime(2025, 2, 21, 22, 24, 40, 186205))
(2, 'María García', 34, 'maria@example.com', datetime.datetime(2025, 2, 21, 22, 24, 40, 186205))
(3, 'Carlos López', 25, 'carlos@example.com', datetime.datetime(2025, 2, 21, 22, 24, 40, 186205))
(4, 'Juan Pérez', 28, 'juan@example.com', datetime.datetime(2025, 2, 21, 22, 36, 20, 519548))
(5, 'María García', 34, 'maria@example.com', datetime.datetime(2025, 2, 21, 22, 36, 20, 519548))
(6, 'Carlos López', 25, 'carlos@example.com', datetime.datetime(2025, 2, 21, 22, 36, 20, 519548))
(7, 'Juan Pérez', 28, 'juan@example.com', datetime.datetime(2025, 2, 21, 22, 36, 22, 326037))
(8, 'María García', 34, 'maria@example.com', datetime.datetime(2025, 2, 21, 22, 36, 22, 326037))
(9, 'Carlos López', 25, 'carlos@example.com', datetime.datetime(2025, 2, 21, 22, 36, 22, 326037))
(10, 'Juan Pérez', 28, 'juan@example.com', datetime.datetime(2025, 2, 21, 22, 36, 25, 133764))
(11, 'María Garc

In [27]:
with engine.connect() as conn:
    df = pd.read_sql("SELECT * FROM tabla_etl", conn)

In [28]:
df

,id,nombre,edad,email,fecha_registro
0,1,Juan Pérez,28,juan@example.com,2025-02-21 22:24:40.186205
1,2,María García,34,maria@example.com,2025-02-21 22:24:40.186205
2,3,Carlos López,25,carlos@example.com,2025-02-21 22:24:40.186205
3,4,Juan Pérez,28,juan@example.com,2025-02-21 22:36:20.519548
4,5,María García,34,maria@example.com,2025-02-21 22:36:20.519548
5,6,Carlos López,25,carlos@example.com,2025-02-21 22:36:20.519548
6,7,Juan Pérez,28,juan@example.com,2025-02-21 22:36:22.326037
7,8,María García,34,maria@example.com,2025-02-21 22:36:22.326037
8,9,Carlos López,25,carlos@example.com,2025-02-21 22:36:22.326037
9,10,Juan Pérez,28,juan@example.com,2025-02-21 22:36:25.133764


In [32]:
df_transformed = df.copy()
df_transformed["nombre"] = df_transformed["nombre"].str.upper()  # Uppercase names
df_transformed["age_category"] = df_transformed["edad"].apply(
    lambda x: "Young" if x < 30 else "Middle-aged" if x < 50 else "Senior"
)

# Display transformed DataFrame
df_transformed.head()

,id,nombre,edad,email,fecha_registro,age_category
0,1,JUAN PÉREZ,28,juan@example.com,2025-02-21 22:24:40.186205,Young
1,2,MARÍA GARCÍA,34,maria@example.com,2025-02-21 22:24:40.186205,Middle-aged
2,3,CARLOS LÓPEZ,25,carlos@example.com,2025-02-21 22:24:40.186205,Young
3,4,JUAN PÉREZ,28,juan@example.com,2025-02-21 22:36:20.519548,Young
4,5,MARÍA GARCÍA,34,maria@example.com,2025-02-21 22:36:20.519548,Middle-aged


In [33]:
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS transformed_data (
            id SERIAL PRIMARY KEY,
            nombre VARCHAR(100),
            edad INT,
            email VARCHAR(100),
            fecha_registro TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            age_category VARCHAR(20)
        );
    """))
    conn.commit()  # Asegúrate de confirmar los cambios
    print("Tabla 'transformed_data' creada exitosamente en PostgreSQL.")

Tabla 'transformed_data' creada exitosamente en PostgreSQL.


In [34]:
with engine.connect() as conn:
    df_transformed.to_sql("transformed_etl", con=engine, if_exists="append", index=False)

print("Transformed data stored successfully in 'transformed_etl'.")

Transformed data stored successfully in 'transformed_etl'.


In [35]:
with engine.connect() as conn:
    db_transformed_df = pd.read_sql("SELECT * FROM transformed_etl", conn)
    
db_transformed_df

,id,nombre,edad,email,fecha_registro,age_category
0,1,JUAN PÉREZ,28,juan@example.com,2025-02-21 22:24:40.186205,Young
1,2,MARÍA GARCÍA,34,maria@example.com,2025-02-21 22:24:40.186205,Middle-aged
2,3,CARLOS LÓPEZ,25,carlos@example.com,2025-02-21 22:24:40.186205,Young
3,1,JUAN PÉREZ,28,juan@example.com,2025-02-21 22:24:40.186205,Young
4,2,MARÍA GARCÍA,34,maria@example.com,2025-02-21 22:24:40.186205,Middle-aged
5,3,CARLOS LÓPEZ,25,carlos@example.com,2025-02-21 22:24:40.186205,Young
6,4,JUAN PÉREZ,28,juan@example.com,2025-02-21 22:36:20.519548,Young
7,5,MARÍA GARCÍA,34,maria@example.com,2025-02-21 22:36:20.519548,Middle-aged
8,6,CARLOS LÓPEZ,25,carlos@example.com,2025-02-21 22:36:20.519548,Young
9,7,JUAN PÉREZ,28,juan@example.com,2025-02-21 22:36:22.326037,Young
